<a href="https://colab.research.google.com/github/andresalcaraaz/Netflix-Project-2-ETL-Data/blob/main/PROJECT_2_ETL_FINAL_FILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://pp

In [5]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

In [5]:
df.createOrReplaceTempView('names')
df.show()

+---------+------------------+---------+---------+--------------------+--------------------+---------+
|   nconst|       primaryname|birthyear|deathyear|   primaryprofession|      knownfortitles| knownFor|
+---------+------------------+---------+---------+--------------------+--------------------+---------+
|nm0458594|     Roger Kleiber|     null|     null|visual_effects,ed...|tt0405022,tt19579...|tt0497467|
|nm0458601|        Mark Kleid|     null|     null|               actor|tt0105041,tt01120...|tt0277615|
|nm0458616|      Mark Kleiman|     null|     null|editor,producer,m...|tt0196091,tt01256...|tt0972846|
|nm0458617|   Michael Kleiman|     1969|     null|camera_department...|tt0913354,tt04311...|tt0431197|
|nm0458618|      Naum Kleiman|     1937|     null|director,actor,wr...|tt3066262,tt64875...|tt3066262|
|nm0458648|     Andreas Klein|     null|     null|producer,executiv...|tt0311429,tt03139...|tt0311429|
|nm0458656|      Arthur Klein|     null|     null|editorial_departm...|tt

In [6]:
###################### Requirement 1 #############################
spark.sql(
"""
select DISTINCT primaryname
      ,primaryprofession
      ,birthyear
      ,deathyear
      ,(year(CURRENT_DATE()) - birthyear) as age
      ,deathyear
      ,case when deathyear is null and (year(CURRENT_DATE()) - birthyear) < 120 then true
            else false
       end as is_alive
  from names
 where (primaryprofession like '%actor%'
       or primaryprofession like '%actress%') 
""").show(20)

+--------------------+--------------------+---------+---------+----+---------+--------+
|         primaryname|   primaryprofession|birthyear|deathyear| age|deathyear|is_alive|
+--------------------+--------------------+---------+---------+----+---------+--------+
|Marcela Kloosterboer|  actress,soundtrack|     1983|     null|38.0|     null|    true|
|        Sylvia Klein|             actress|     null|     null|null|     null|   false|
|     Walter Klenhard|writer,actor,dire...|     null|     null|null|     null|   false|
|       Nick Raffaele|actor,sound_depar...|     null|     null|null|     null|   false|
|       Stef Zamorano|             actress|     null|     null|null|     null|   false|
|         Kevin Beyer|               actor|     null|     null|null|     null|   false|
|       Vincent Mosso|               actor|     null|     null|null|     null|   false|
|       Sofia Marques|actress,director,...|     null|     null|null|     null|   false|
|    Rebecca Charnley|          

In [7]:
####################### Requirement 2 ##################################
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/titles_basic.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("titles_basic.csv"), sep=",", header=True)

In [8]:
df.createOrReplaceTempView('names2')

In [9]:
df2 = spark.sql("SELECT * FROM names2")
clean_professions_df=df2.drop("genres","primarytitle","isadult", "startyear", "endyear", "_c0")
clean_professions_df.show()

+---------+---------+--------------------+--------------+
|   tconst|titletype|       originaltitle|runtimeminutes|
+---------+---------+--------------------+--------------+
|tt0439997|    movie|           500 Almas|         105.0|
|tt0439999|tvSpecial|           80s Mania|          50.0|
|tt0440003|    movie|         A1 tou tiao|          95.0|
|tt0440004|  tvMovie| AD/BC: A Rock Opera|          30.0|
|tt0440008|  tvMovie|Abbamania: We Say...|          50.0|
|tt0440016|    movie|       Ah ma yau nan|          93.0|
|tt0440022|  tvMovie|        Al atardecer|          66.0|
|tt0440035|  tvMovie|      L'amour en pen|          52.0|
|tt0440067|    movie|      Bau lit do see|          99.0|
|tt0440078|  tvMovie|  The Band Aid Story|          95.0|
|tt0440084|  tvMovie|A Beachcombers Ch...|         120.0|
|tt0440149|    video|Blink 182: Punk P...|          61.0|
|tt0440154|  tvMovie|Boogie special: 5...|          26.0|
|tt0440155|  tvMovie|Boogie special: M...|          28.0|
|tt0440157|tvS

In [10]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/ratings.csv"
spark.sparkContext.addFile(url)
df2 = spark.read.csv(SparkFiles.get("ratings.csv"), sep=",", header=True)

In [11]:
df2.createOrReplaceTempView('ratings')

In [12]:
df2 = spark.sql("SELECT * FROM ratings")
clean_ratings_df=df2.drop("numvotes")
clean_ratings_df.show()

+---------+-------------+
|   tconst|averagerating|
+---------+-------------+
|tt0214461|          7.4|
|tt0214659|          6.0|
|tt0214878|          6.1|
|tt0215244|          3.0|
|tt0215402|          8.1|
|tt0215423|          8.1|
|tt0215458|          5.6|
|tt0215785|          5.3|
|tt0215972|          5.5|
|tt0216247|          8.1|
|tt0216705|          7.0|
|tt0217135|          5.9|
|tt0217143|          6.5|
|tt0217196|          7.4|
|tt0217629|          7.0|
|tt0217769|          5.8|
|tt0217824|          6.3|
|tt0217836|          2.5|
|tt0218182|          6.3|
|tt0218440|          6.0|
+---------+-------------+
only showing top 20 rows



In [13]:
complete_df = clean_professions_df.join(clean_ratings_df, on = ['tconst'])
complete_df.show()

+---------+---------+--------------------+--------------+-------------+
|   tconst|titletype|       originaltitle|runtimeminutes|averagerating|
+---------+---------+--------------------+--------------+-------------+
|tt0439997|    movie|           500 Almas|         105.0|          7.2|
|tt0440003|    movie|         A1 tou tiao|          95.0|          5.9|
|tt0440004|  tvMovie| AD/BC: A Rock Opera|          30.0|          7.4|
|tt0440016|    movie|       Ah ma yau nan|          93.0|          5.5|
|tt0440067|    movie|      Bau lit do see|          99.0|          5.4|
|tt0440078|  tvMovie|  The Band Aid Story|          95.0|          8.1|
|tt0440084|  tvMovie|A Beachcombers Ch...|         120.0|          7.0|
|tt0440149|    video|Blink 182: Punk P...|          61.0|          4.1|
|tt0440154|  tvMovie|Boogie special: 5...|          26.0|          1.0|
|tt0440155|  tvMovie|Boogie special: M...|          28.0|          7.7|
|tt0440157|tvSpecial|The British Comed...|         125.0|       

In [21]:
from datetime import timedelta
from pyspark.sql.functions import udf
def minutesToHourMinutes(minuteString = "0.0"):
  
  
  if minuteString:
    minutesAsFloat = float(minuteString)
    return str(timedelta(minutes=minutesAsFloat))[:-3]
  else:
    return "0:0"

minutesToHourMinutesUdf = udf(lambda z: minutesToHourMinutes(z), StringType())
spark.udf.register("minutesToHourMinutesUdf", minutesToHourMinutesUdf)

newTitleDf = complete_df.withColumn("formatted_hhmm", minutesToHourMinutesUdf("runtimeminutes"))
newTitleDf.show(10)

+---------+---------+--------------------+--------------+-------------+--------------+
|   tconst|titletype|       originaltitle|runtimeminutes|averagerating|formatted_hhmm|
+---------+---------+--------------------+--------------+-------------+--------------+
|tt0439997|    movie|           500 Almas|         105.0|          7.2|          1:45|
|tt0440003|    movie|         A1 tou tiao|          95.0|          5.9|          1:35|
|tt0440004|  tvMovie| AD/BC: A Rock Opera|          30.0|          7.4|          0:30|
|tt0440016|    movie|       Ah ma yau nan|          93.0|          5.5|          1:33|
|tt0440067|    movie|      Bau lit do see|          99.0|          5.4|          1:39|
|tt0440078|  tvMovie|  The Band Aid Story|          95.0|          8.1|          1:35|
|tt0440084|  tvMovie|A Beachcombers Ch...|         120.0|          7.0|          2:00|
|tt0440149|    video|Blink 182: Punk P...|          61.0|          4.1|          1:01|
|tt0440154|  tvMovie|Boogie special: 5...| 

In [15]:
##################### Requirement 3 #################################

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(url)
fp_df = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

# Show DataFrame
fp_df.show()

+----------+--------+---------+---------------+----+-----------------+
|    tconst|ordering|   nconst|       category| job|       characters|
+----------+--------+---------+---------------+----+-----------------+
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt121

In [16]:
fp_df.orderBy(fp_df["category"].desc()).show()

+----------+--------+----------+--------+--------------------+----------+
|    tconst|ordering|    nconst|category|                 job|characters|
+----------+--------+----------+--------+--------------------+----------+
| tt3257624|       6| nm6026527|  writer|        story editor|      null|
| tt1396634|       2| nm1112737|  writer|              writer|      null|
| tt3281548|       6| nm0017301|  writer|based on the nove...|      null|
|tt12204050|       8|nm11523934|  writer|                null|      null|
| tt3281548|       6| nm0017301|  writer|based on the nove...|      null|
| tt1220717|       6| nm1658333|  writer|based on the nove...|      null|
| tt3281548|       6| nm0017301|  writer|based on the nove...|      null|
| tt1220717|       6| nm1658333|  writer|based on the nove...|      null|
| tt3281548|       6| nm0017301|  writer|based on the nove...|      null|
| tt9782790|       5| nm3744684|  writer|       script writer|      null|
| tt1396525|       6| nm1771584|  writ

In [17]:
fp_df.filter(fp_df.job == "producer").show(truncate=False)

+----------+--------+----------+--------+--------+----------+
|tconst    |ordering|nconst    |category|job     |characters|
+----------+--------+----------+--------+--------+----------+
|tt12193904|5       |nm11519930|producer|producer|null      |
|tt12193904|5       |nm11519930|producer|producer|null      |
|tt12193904|5       |nm11519930|producer|producer|null      |
|tt12193904|5       |nm11519930|producer|producer|null      |
|tt12198108|5       |nm11519930|producer|producer|null      |
|tt12198108|5       |nm11519930|producer|producer|null      |
|tt12198108|5       |nm11519930|producer|producer|null      |
|tt12198108|5       |nm11519930|producer|producer|null      |
|tt9234450 |9       |nm3882588 |producer|producer|null      |
|tt9234450 |9       |nm3882588 |producer|producer|null      |
|tt9234450 |9       |nm3882588 |producer|producer|null      |
|tt9234450 |9       |nm3882588 |producer|producer|null      |
|tt3263638 |5       |nm0588711 |producer|producer|null      |
|tt32636

In [18]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(url)
fn_df = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

# Show DataFrame
fn_df.show()

+---------+------------------+---------+---------+--------------------+--------------------+---------+
|   nconst|       primaryname|birthyear|deathyear|   primaryprofession|      knownfortitles| knownFor|
+---------+------------------+---------+---------+--------------------+--------------------+---------+
|nm0458594|     Roger Kleiber|     null|     null|visual_effects,ed...|tt0405022,tt19579...|tt0497467|
|nm0458601|        Mark Kleid|     null|     null|               actor|tt0105041,tt01120...|tt0277615|
|nm0458616|      Mark Kleiman|     null|     null|editor,producer,m...|tt0196091,tt01256...|tt0972846|
|nm0458617|   Michael Kleiman|     1969|     null|camera_department...|tt0913354,tt04311...|tt0431197|
|nm0458618|      Naum Kleiman|     1937|     null|director,actor,wr...|tt3066262,tt64875...|tt3066262|
|nm0458648|     Andreas Klein|     null|     null|producer,executiv...|tt0311429,tt03139...|tt0311429|
|nm0458656|      Arthur Klein|     null|     null|editorial_departm...|tt

In [19]:
fn_df.filter(fn_df.primaryprofession == "producer").show(truncate=False)

+---------+------------------+---------+---------+-----------------+----------------------------------------+----------+
|nconst   |primaryname       |birthyear|deathyear|primaryprofession|knownfortitles                          |knownFor  |
+---------+------------------+---------+---------+-----------------+----------------------------------------+----------+
|nm0460805|Arlene Knight     |null     |null     |producer         |tt4524446,tt4082122,tt3685368,tt2728252 |tt4524446 |
|nm0462049|Sachiko Kobayashi |null     |null     |producer         |tt5498350,tt0092963,tt0234669,tt6413988 |tt6413988 |
|nm0459057|Katharina Kleine  |null     |null     |producer         |tt0108979,tt0196264,tt0235930,tt0106178 |tt0235930 |
|nm0459354|Daniel S. Kletzky |null     |null     |producer         |tt0317676,tt0120804,tt0173840,tt0398913 |tt0120804 |
|nm0459859|Alex Klippel      |null     |null     |producer         |tt0333986,tt0259853,tt0257986,tt2353967 |tt0257986 |
|nm0459275|Hartmut Klenke    |nu

In [20]:
fn_df.drop('nconst','birthyear','deathyear','knownfortitles','knownFor').filter(fn_df.primaryprofession == "producer").show(truncate=False)

+------------------+-----------------+
|primaryname       |primaryprofession|
+------------------+-----------------+
|Arlene Knight     |producer         |
|Sachiko Kobayashi |producer         |
|Katharina Kleine  |producer         |
|Daniel S. Kletzky |producer         |
|Alex Klippel      |producer         |
|Hartmut Klenke    |producer         |
|Daniel S. Kletzky |producer         |
|Shantel Klinger   |producer         |
|Patrick Lam       |producer         |
|Morihiro Kodama   |producer         |
|Mari Bonnemaison  |producer         |
|Teennick Top10    |producer         |
|Dee Smith         |producer         |
|Max Arvelaiz      |producer         |
|Chantal Rickards  |producer         |
|Mark L. Smith     |producer         |
|Daeun Kim         |producer         |
|Jeffrey Pechter   |producer         |
|Mo El Hami        |producer         |
|Jean-Claude Sercer|producer         |
+------------------+-----------------+
only showing top 20 rows



In [13]:
#################### Requirement 4 ##########################
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(url)
character = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [14]:
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(url)
film = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

In [15]:
# Save CSV into DataFrame
character = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [16]:
# Save CSV into DataFrame
character = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)
# clean drop tconstant, job, ordering
character = character.drop("tconst","job","ordering")
#  clean film drop deathyear and birthyear
film = film.drop("deathyear","birthyear")
#join tables on nconstant
new = character.join(film, "nconst", "outer")
# filter dataframe to only actor 
newest = new.filter(new["category"] == "actor").show()
#drop null values
final = newest.dropna(how="any").show(truncate=False)


+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|   nconst|category|          characters|primaryname|   primaryprofession|      knownfortitles| knownFor|
+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary 

AttributeError: ignored

In [17]:
film = film.drop("deathyear","birthyear")
film.show()

+---------+------------------+--------------------+--------------------+---------+
|   nconst|       primaryname|   primaryprofession|      knownfortitles| knownFor|
+---------+------------------+--------------------+--------------------+---------+
|nm0458594|     Roger Kleiber|visual_effects,ed...|tt0405022,tt19579...|tt0497467|
|nm0458601|        Mark Kleid|               actor|tt0105041,tt01120...|tt0277615|
|nm0458616|      Mark Kleiman|editor,producer,m...|tt0196091,tt01256...|tt0972846|
|nm0458617|   Michael Kleiman|camera_department...|tt0913354,tt04311...|tt0431197|
|nm0458618|      Naum Kleiman|director,actor,wr...|tt3066262,tt64875...|tt3066262|
|nm0458648|     Andreas Klein|producer,executiv...|tt0311429,tt03139...|tt0311429|
|nm0458656|      Arthur Klein|editorial_departm...|tt0106925,tt13375...|tt1337579|
|nm0458664|        Bart Klein|               actor|tt0830854,tt03064...|tt0306413|
|nm0458674|Bonnie Sherr Klein|director,producer...|tt0227531,tt02242...|tt5492464|
|nm0

In [18]:
new = character.join(film, "nconst", "outer")


In [19]:
#sort on character names 
new.show()

+---------+---------------+--------------------+-----------+-----------------+--------------+--------+
|   nconst|       category|          characters|primaryname|primaryprofession|knownfortitles|knownFor|
+---------+---------------+--------------------+-----------+-----------------+--------------+--------+
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             

In [20]:
# filter dataframe to only actor and actresses
newest = new.filter(new["category"] == "actor").show()

+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|   nconst|category|          characters|primaryname|   primaryprofession|      knownfortitles| knownFor|
+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary 